# Importing all the required modules
---

In [ ]:
import numpy as np
import pandas as pd
import torch

# Loading the trainng Data !
---

In [ ]:
data = pd.read_csv('train.csv')
train_list = []
for i in range(len(data)):
    if(data['clickbait'][i] ==1):
        train_list.append(data['headline'][i])

# Analysing and stroring the bigrams just for future reference

---

In [ ]:
sentences = train_list[:int(0.20*len(train_list))]
b = {}
for sent in sentences:
  chs = ['<S>'] + sent.split() + ['<E>']
  for ch1, ch2 in zip(chs, chs[1:]):
    bigram = (ch1, ch2)
    b[bigram] = b.get(bigram, 0) + 1
sorted(b.items(), key = lambda kv: -kv[1])

# Constructing the strings_to_index and inverse dictionaries
---

In [ ]:
words = sorted(list(set((' '.join(sentences)).split())))
stoi = {s:i+1 for i,s in enumerate(words)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

# Preparing the training data for our neural network
---

In [ ]:
xs, ys = [], []

for sent in sentences:
  chs = ['.'] + sent.split() + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

# construting the generator object and training the on the data!
---

In [ ]:
import torch.nn.functional as F

num = xs.nelement()
g = torch.Generator().manual_seed(1234)

W = torch.randn((len(stoi),len(stoi)),generator=g , requires_grad = True)

for k in range(90):

  # forward pass
  xenc = F.one_hot(xs, num_classes=len(stoi)).float() # input to the network: one-hot encoding
  logits = xenc @ W # predict log-counts
  counts = logits.exp() # counts, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
  loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
  print(loss.item())
  W.grad = None # set to zero the gradient
  loss.backward()

  # update
  W.data += -1200 * W.grad

# Storing the the trained model
---

In [ ]:
import joblib
txt = 1234
joblib.dump(txt,'seed.sav')
joblib.dump(stoi,'stoi.sav')
joblib.dump(itos,'itos.sav')